## IMPORTING DATA TO QUEST TIMESERIES DATABASE USING PYTHON AND IT'S LIBRARIES

In [43]:
#importing libraries
import pandas as pd
import requests
from io import StringIO

import psycopg2

data = pd.read_csv("cropped.csv")

# Constituency Data
constituencies = [
    {"constituencyID": 1, "constituency": "Bristol East", "Member of Parliament": "Kerry McCarthy"},
    {"constituencyID": 2, "constituency": "Bristol Northwest", "Member of Parliament": "Darren Jones"},
    {"constituencyID": 3, "constituency": "Bristol South", "Member of Parliament": "Karin Smyth"},
    {"constituencyID": 4, "constituency": "Bristol West", "Member of Parliament": "Thangam Debbonaire"}
]

# Site Data
site_data = [
    {"SiteID": 188, "Site name": "AURN Bristol Centre", "Latitude": 51.4572041156, "Longitude": -2.58564914143, "CONSTITUENCY_constituencyID": 1},
    {"SiteID": 203, "Site name": "Brislington Depot", "Latitude": 51.4417471802, "Longitude": -2.55995583224, "CONSTITUENCY_constituencyID": 1},
    {"SiteID": 206, "Site name": "Rupert Street", "Latitude": 51.4554331987, "Longitude": -2.59626237324, "CONSTITUENCY_constituencyID": 2},
    {"SiteID": 209, "Site name": "IKEA M32", "Latitude": 51.4752847609, "Longitude": -2.56207998299, "CONSTITUENCY_constituencyID": 2},
    {"SiteID": 213, "Site name": "Old Market", "Latitude": 51.4560189999, "Longitude": -2.58348949026, "CONSTITUENCY_constituencyID": 3},
    {"SiteID": 215, "Site name": "Parson Street School", "Latitude": 51.432675707, "Longitude": -2.60495665673, "CONSTITUENCY_constituencyID": 3},
    {"SiteID": 228, "Site name": "Temple Meads Station", "Latitude": 51.4488837041, "Longitude": -2.58447776241, "CONSTITUENCY_constituencyID": 4},
    {"SiteID": 270, "Site name": "Wells Road", "Latitude": 51.4278638883, "Longitude": -2.56374153315, "CONSTITUENCY_constituencyID": 4},
    {"SiteID": 271, "Site name": "Trailer Portway P&R", "Latitude": 51.4899934596, "Longitude": -2.68877856929, "CONSTITUENCY_constituencyID": 1},
    {"SiteID": 375, "Site name": "Newfoundland Road Police Station", "Latitude": 51.4606738207, "Longitude": -2.58225341824, "CONSTITUENCY_constituencyID": 2},
    {"SiteID": 395, "Site name": "Shiner's Garage", "Latitude": 51.4577930324, "Longitude": -2.56271419977, "CONSTITUENCY_constituencyID": 3},
    {"SiteID": 452, "Site name": "AURN St Pauls", "Latitude": 51.4628294172, "Longitude": -2.58454081635, "CONSTITUENCY_constituencyID": 4},
    {"SiteID": 447, "Site name": "Bath Road", "Latitude": 51.4425372726, "Longitude": -2.57137536073, "CONSTITUENCY_constituencyID": 1},
    {"SiteID": 459, "Site name": "Cheltenham Road \\ Station Road", "Latitude": 51.4689385901, "Longitude": -2.5927241667, "CONSTITUENCY_constituencyID": 2},
    {"SiteID": 463, "Site name": "Fishponds Road", "Latitude": 51.4780449714, "Longitude": -2.53523027459, "CONSTITUENCY_constituencyID": 3},
    {"SiteID": 481, "Site name": "CREATE Centre Roof", "Latitude": 51.447213417, "Longitude": -2.62247405516, "CONSTITUENCY_constituencyID": 4},
    {"SiteID": 500, "Site name": "Temple Way", "Latitude": 51.4579497132, "Longitude": -2.5839890903, "CONSTITUENCY_constituencyID": 1},
    {"SiteID": 501, "Site name": "Colston Avenue", "Latitude": 51.4552693827, "Longitude": -2.59664882855, "CONSTITUENCY_constituencyID": 2},
    {"SiteID": 672, "Site name": "Marlborough Street", "Latitude": 51.4591419717, "Longitude": -2.59543271836, "CONSTITUENCY_constituencyID": 3}
]

## FLATTENING OUR TABLES INTO ONE(DENORMALISED FORMAT) SO THAT WE CAN USE A NOSQL DATABASE

In [44]:
import pandas as pd

# Load data from CSV files
constituency_df = pd.DataFrame(constituencies)
sites_df = pd.DataFrame(site_data)
sample_df = pd.read_csv("sample_data.csv")

# Merge observation with site data
merged_df = pd.merge(data, sites_df, left_on="Site_ID", right_on="SiteID", how="left")

# Merge the result with constituency data
denormalized_df = pd.merge(merged_df, constituency_df, left_on="CONSTITUENCY_constituencyID", right_on="constituencyID", how="left")

In [45]:
denormalized_df.head(20)

,ObservationID,Date_Time,Site_ID,NOx,NO2,NO,PM10,O3,Temperature,ObjectId,...,Pressure,SO2,SiteID,Site name,Latitude,Longitude,CONSTITUENCY_constituencyID,constituencyID,constituency,Member of Parliament
0,0,2023-10-22 14:00:00,672.0,31.745374,17.305761,9.507336,NaN,NaN,NaN,NaN,...,NaN,NaN,672,Marlborough Street,51.459142,-2.595433,3,3,Bristol South,Karin Smyth
1,1,2023-10-22 14:00:00,463.0,54.438782,24.058720,19.790930,NaN,NaN,NaN,NaN,...,NaN,NaN,463,Fishponds Road,51.478045,-2.535230,3,3,Bristol South,Karin Smyth
2,2,2023-10-22 14:00:00,270.0,68.668313,23.036062,29.792909,NaN,NaN,NaN,NaN,...,NaN,NaN,270,Wells Road,51.427864,-2.563742,4,4,Bristol West,Thangam Debbonaire
3,3,2023-10-22 14:00:00,501.0,132.894844,50.758813,53.478635,NaN,NaN,NaN,NaN,...,NaN,NaN,501,Colston Avenue,51.455269,-2.596649,2,2,Bristol Northwest,Darren Jones
4,4,2023-10-22 14:00:00,203.0,28.533969,14.852156,8.924710,NaN,NaN,NaN,NaN,...,NaN,NaN,203,Brislington Depot,51.441747,-2.559956,1,1,Bristol East,Kerry McCarthy
5,5,2023-10-22 14:00:00,215.0,64.195719,29.552375,22.591483,NaN,NaN,NaN,NaN,...,NaN,NaN,215,Parson Street School,51.432676,-2.604957,3,3,Bristol South,Karin Smyth
6,6,2023-10-22 13:00:00,270.0,67.187720,19.775782,31.004577,NaN,NaN,NaN,NaN,...,NaN,NaN,270,Wells Road,51.427864,-2.563742,4,4,Bristol West,Thangam Debbonaire
7,7,2023-10-22 13:00:00,463.0,28.979156,15.844000,8.562733,NaN,NaN,NaN,NaN,...,NaN,NaN,463,Fishponds Road,51.478045,-2.535230,3,3,Bristol South,Karin Smyth
8,8,2023-10-22 13:00:00,215.0,58.121937,27.166532,20.185467,NaN,NaN,NaN,NaN,...,NaN,NaN,215,Parson Street School,51.432676,-2.604957,3,3,Bristol South,Karin Smyth
9,9,2023-10-22 13:00:00,672.0,36.851402,16.654877,13.102427,NaN,NaN,NaN,NaN,...,NaN,NaN,672,Marlborough Street,51.459142,-2.595433,3,3,Bristol South,Karin Smyth


Our tables have been flattened and we now have a single table with all the data. The table has redundant columns and we will have to drop them since it is meaningless to have two separate columns communicating the same thing. This makes our data more clean and easier to work with.

In [46]:
# Drop redundant columns if necessary
denormalized_df = denormalized_df.drop(columns=["Site_ID", "CONSTITUENCY_constituencyID"])
#renaming column names with spaces
denormalized_df.columns = denormalized_df.columns.str.replace(" ", "_")
#sorting data by datetime
denormalized_df = denormalized_df.sort_values(by='Date_Time', ascending=True)

In [47]:
denormalized_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 520833 entries, 520832 to 0
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ObservationID         520833 non-null  int64  
 1   Date_Time             520833 non-null  object 
 2   NOx                   493537 non-null  float64
 3   NO2                   493567 non-null  float64
 4   NO                    493532 non-null  float64
 5   PM10                  131522 non-null  float64
 6   O3                    69647 non-null   float64
 7   Temperature           107573 non-null  float64
 8   ObjectId              0 non-null       float64
 9   ObjectId2             520833 non-null  int64  
 10  NVPM10                24710 non-null   float64
 11  VPM10                 24710 non-null   float64
 12  NVPM2_5               23580 non-null   float64
 13  PM2_5                 94685 non-null   float64
 14  VPM2_5                23580 non-null   float64
 15  CO   

After merging the three tables into one, we still have the same number of records we had initially when they were separate. Since we only need to model one station to the `NoSQl` database, we are going to filter the data to only include the records for the station we choose. To my preference I choose `AURN St Pauls` station of siteID `452`

In [48]:
#filtering data to only have data from AURN
station_data = denormalized_df[denormalized_df["SiteID"] == 452]
#looking at the first 5 records
station_data.head()

,ObservationID,Date_Time,NOx,NO2,NO,PM10,O3,Temperature,ObjectId,ObjectId2,...,RH,Pressure,SO2,SiteID,Site_name,Latitude,Longitude,constituencyID,constituency,Member_of_Parliament
520829,520829,2015-01-01 00:00:00,11.331560,10.039350,0.84276,28.100000,53.152142,NaN,NaN,1041709,...,NaN,NaN,NaN,452,AURN St Pauls,51.462829,-2.584541,4,Bristol West,Thangam Debbonaire
520825,520825,2015-01-01 01:00:00,15.164850,12.742990,1.57950,22.700001,50.241749,NaN,NaN,1378080,...,NaN,NaN,NaN,452,AURN St Pauls,51.462829,-2.584541,4,Bristol West,Thangam Debbonaire
520816,520816,2015-01-01 02:00:00,19.372190,16.950491,1.57939,22.299999,46.948841,NaN,NaN,240085,...,NaN,NaN,NaN,452,AURN St Pauls,51.462829,-2.584541,4,Bristol West,Thangam Debbonaire
520808,520808,2015-01-01 03:00:00,16.747761,13.962210,1.81669,25.500000,51.489059,NaN,NaN,630428,...,NaN,NaN,NaN,452,AURN St Pauls,51.462829,-2.584541,4,Bristol West,Thangam Debbonaire
520802,520802,2015-01-01 04:00:00,18.372910,12.680830,3.71228,21.799999,53.983688,NaN,NaN,1041703,...,NaN,NaN,NaN,452,AURN St Pauls,51.462829,-2.584541,4,Bristol West,Thangam Debbonaire


We are going to take sample data of the station and model it into our database. We will use the first 20000 records of the station to populate our database.

In [49]:
#first 20000 records
station_sample = station_data.head(20000)
#saving to csv file
station_sample.to_csv('station_sample.csv', index=False)

In [51]:
import psycopg2
import pandas as pd

# Path to your cropped CSV file
csv_file_path = "station_sample.csv"

# Load the CSV into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Connect to QuestDB's PostgreSQL wire protocol
conn = psycopg2.connect(
    host="localhost",
    port=8812,  # Default port for QuestDB's PostgreSQL interface
    database="qdb",  # Default database name
    user="admin",
    password="quest"
)

cursor = conn.cursor()

# Insert data row-by-row
insert_query = """
INSERT INTO STATION_DATA (ObservationID, Date_time, NOx, NO2, NO, PM10, O3, Temperature,ObjectID, ObjectID2,
NVPM10, VPM10, NVPM2_5, VPM2_5, CO, RH, Pressure, SO2, SiteID, Site_name, Latitude, Longitude, constituencyID, constituency,
Member_of_Parliament)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s);
"""

for _, row in df.iterrows():
    cursor.execute(insert_query,(
        row["ObservationID"],
        row["Date_Time"],
        row["NOx"],
        row["NO2"],
        row["NO"],
        row["PM10"],
        row["O3"],
        row["Temperature"],
        row["ObjectId"],
        row["ObjectId2"],
        row["NVPM10"],
        row["VPM10"],
        row["NVPM2_5"],
        row["VPM2_5"],
        row["CO"],
        row["RH"],
        row["Pressure"],
        row["SO2"],
        row["SiteID"],
        row["Site_name"],
        row["Latitude"],
        row["Longitude"],
        row["constituencyID"],
        row["constituency"],
        row["Member_of_Parliament"]
        
        ))

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully inserted into QuestDB!")



Data successfully inserted into QuestDB!
